Import libraries and set url to scrape

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

from sqlalchemy import create_engine, Table, Column, Integer, String, DateTime, text, select, insert, and_, or_
from sqlalchemy.orm import sessionmaker,  DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy.engine import URL
import pyodbc

from typing import List, Optional
from unidecode import unidecode
import pandas as pd
import numpy as np
import re
import time

SERVER = 'localhost\SQLEXPRESS'
DRIVER = '{ODBC Driver 18 for SQL Server}'
DATABASE = 'nba_standings'
nba_standings_db_string = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};TrustServerCertificate=yes;Trusted_Connection=yes'
nba_standings_connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": nba_standings_db_string})

In [2]:
def scrape_nba_playoff_data(url, last_year=2023):
    html = urlopen(url)      
    soup = BeautifulSoup(html, features="lxml")

    # use getText()to extract the headers into a list
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    rows = soup.findAll('tr')[2:]
    rows_data = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]
    # if you print row_data here you'll see an empty row
    # so, remove the empty row
    rows_data.pop(20)
    # for simplicity subset the data for only 39 seasons
    rows_data = rows_data[0:38]
    # we're missing a column for years
    # add the years into rows_data
    for i in range(0, len(rows_data)):
        rows_data[i].insert(0, last_year)
        last_year -=1

    # create the dataframe
    nba_finals = pd.DataFrame(rows_data, columns = headers)
    
    # Separate player and valye for playoff leaders
    player_pattern = re.compile('([a-zA-Z. ]+) \(')
    count_pattern = re.compile('[0-9.]{2,4}')
    for stat in ['Points', 'Rebounds', 'Assists', 'Win Shares']:

        player = [re.findall(player_pattern, unidecode(pts))[0] for pts in nba_finals_df[stat]]
        count = [re.findall(count_pattern, unidecode(pts))[0] for pts in nba_finals_df[stat]]

        nba_finals_df[f'{stat} Leader'] = player
        nba_finals_df[stat] = count

    return nba_finals

In [3]:
# create a function to scrape team performance for multiple years
def scrape_NBA_team_data(years = [2017, 2018]):
    
    final_df = pd.DataFrame(columns = ["Year", "Team", "W", "L",
                                       "W/L%", "GB", "PS/G", "PA/G",
                                       "SRS", "Playoffs",
                                       "Losing_season"])
    for year in years:
        # NBA season to scrape
        url = f"https://www.basketball-reference.com/leagues/NBA_{year}_standings.html"
        html = urlopen(url)
        soup = BeautifulSoup(html, features="lxml")
        titles = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

        # first, find only column headers
        seen = set()
        seen_add = seen.add
        headers = [t for t in titles if len(t.split(' ')) < 2 and not (t in seen or seen_add(t))]
        # then, exclude first set of column headers (duplicated)
        seen = set()
        seen_add = seen.add
        titles = [t for t in titles if len(t.split(' ')) >= 2 and 'Division' not in t.split(' ') and 'Conference' not in t.split(' ') and not (t in seen or seen_add(t))]

        # next, grab all data from rows (avoid first row)
        rows = soup.findAll('tr')[1:]
        team_stats = [[td.getText() for td in rows[i].findAll('td')] 
                    for i in range(len(rows))]

        # remove empty elements
        team_stats = [e for e in team_stats if e != []]

        # only keep needed rows
        team_stats = team_stats[0:len(titles)]

        # add team name to each row in team_stats
        for i in range(0, len(team_stats)):
            team_stats[i].insert(0, titles[i])
            team_stats[i].insert(0, year)

        # add team, year columns to headers
        headers.insert(0, "Team")
        headers.insert(0, "Year")

        # create a dataframe with all aquired info
        year_standings = pd.DataFrame(team_stats, columns = headers)

        # add a column to dataframe to indicate playoff appearance
        year_standings["Playoffs"] = ["Y" if "*" in ele else "N" for ele in year_standings["Team"]]
        # remove * from team names
        year_standings["Team"] = [ele.replace('*', '') for ele in year_standings["Team"]]
        # add losing season indicator (win % < .5)
        year_standings["Losing_season"] = ["Y" if float(ele) < .5 else "N" for ele in year_standings["W/L%"]]

        print(f'{year}: {year_standings.shape[0]} teams')

        # append new dataframe to final_df
        final_df = pd.concat((final_df, year_standings), axis=0, ignore_index=True)
    
        time.sleep(10)

    return final_df

In [4]:
def scrape_advanced_stats_data(years = [2017, 2018]):
    
    final_df = pd.DataFrame()

    # for year in years:
        # NBA season to scrape
    for year in years:

        url = f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'
        html = urlopen(url)
        soup = BeautifulSoup(html, features="lxml")
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

        # next, grab all data from rows (avoid first row)
        rows = soup.findAll('tr')[1:]
        player_stats = [[td.getText() for td in rows[i].findAll('td')] 
                        for i in range(len(rows))]

        # remove empty elements
        player_stats = [e for e in player_stats if e != []]

        # add year to each row in player_stats
        for i in range(0, len(player_stats)):
            player_stats[i].insert(0, year)

        # add year column to headers, remove Rk
        headers.insert(0, "Year")
        headers.remove('Rk')

        # create a dataframe with all aquired info
        year_stats = pd.DataFrame(player_stats, columns = headers)
        year_stats = year_stats.drop(labels = '\xa0', axis = 1)

        print(f'{year}: {year_stats.shape[0]} players')

        # append new dataframe to final_df
        final_df = pd.concat((final_df, year_stats), axis=0, ignore_index=True)
    
        time.sleep(10)    

    return final_df

In [ ]:
nba_finals_df = scrape_nba_playoff_data(url=playoff_url, last_year=2023)
nba_finals_df

In [ ]:
nba_standings_data = scrape_NBA_team_data(years = range(2015, 2024))
nba_standings_data


In [ ]:
nba_advanced_stats = scrape_advanced_stats_data(years = range(2015, 2024))
nba_advanced_stats

### Connect to SQL Express database

In [3]:
engine = create_engine(nba_standings_connection_url)

### Push NBA Standings and Advanced Stats dataframes to database

In [ ]:
nba_standings_data.to_sql('nba_standings', engine.engine)

nba_advanced_stats['index'] = range(nba_advanced_stats.shape[0])
bad_columns = [col for col in nba_advanced_stats.columns if col == '\xa0']
if len(bad_columns) > 0:
    nba_advanced_stats = nba_advanced_stats.drop(labels = '\xa0', axis = 1)

nba_advanced_stats.to_sql('nba_advanced_stats', engine.engine)

### Practice Queries in Pandas

In [13]:
query = f"""
    SELECT * 
    FROM nba_standings
    WHERE W > 48   
    ORDER BY W DESC, SRS;
    """  
# query = f"""
#     SELECT * 
#     FROM nba_standings
#     """  

with engine.connect() as conn, conn.begin():
    query = query  
    queried_standings = pd.read_sql_query(query, conn)
queried_standings

,index,Year,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,Playoffs,Losing_season
0,45,2016,Golden State Warriors,73,9,.890,0,114.9,104.1,10.38,Y,N
1,20,2015,Golden State Warriors,67,15,.817,0,110.0,99.9,10.01,Y,N
2,46,2016,San Antonio Spurs,67,15,.817,6.0,103.5,92.9,10.28,Y,N
3,75,2017,Golden State Warriors,67,15,.817,0,115.9,104.3,11.35,Y,N
4,10,2013,Miami Heat,66,16,.805,0,102.9,95.0,7.03,Y,N
...,...,...,...,...,...,...,...,...,...,...,...,...
79,140,2019,Oklahoma City Thunder,49,33,.598,8.0,114.5,111.1,3.56,Y,N
80,123,2019,Boston Celtics,49,33,.598,11.0,112.4,108.0,3.90,Y,N
81,180,2021,Philadelphia 76ers,49,23,.681,0,113.6,108.1,5.28,Y,N
82,229,2022,Utah Jazz,49,33,.598,15.0,113.6,107.6,5.67,Y,N


### Add some years to the standings data

In [ ]:
more_nba_standings_data = scrape_NBA_team_data(years = range(2013, 2015))
with engine.connect() as conn, conn.begin():
    more_nba_standings_data.to_sql('nba_standings', conn, if_exists='append')


### Practice Queries in sqlalchemy

In [ ]:
with Session(bind=engine) as session:
    result = session.execute(
        text("UPDATE nba_standings SET GB=0 WHERE GB LIKE '%\u2014%'"))
    session.commit()

### Grab tables from database using table reflection in SQLAlchemy

In [7]:
class Base(DeclarativeBase):
     pass
class Standings(Base):
    __table__ = Table(
        "nba_standings",
        Base.metadata,
        Column("index", 
                Integer, 
                primary_key=True),
        autoload_with=engine,
    )

# nba_standings
Standings

__main__.Standings

### Push advanced_stats_df to msSQL, then store it as a table in SQLAlchemy

In [ ]:
nba_advanced_stats['index'] = range(nba_advanced_stats.shape[0])
columns = [col for idx, col in enumerate(nba_advanced_stats.columns)]
bad_columns = [(idx, col) for idx, col in enumerate(nba_advanced_stats.columns) if col == '\xa0']
if len(bad_columns) > 0:
    nba_advanced_stats = nba_advanced_stats.drop(labels = '\xa0', axis = 1)

nba_advanced_stats.to_sql('nba_advanced_stats', engine.engine)

In [ ]:

class AdvancedStats(Base):
    __table__ = Table(
        "nba_advanced_stats",
        Base.metadata,
        Column("index", 
                Integer, 
                primary_key=True),
        autoload_with=engine,
    )


In [10]:
AdvancedStats.__table__

Table('nba_advanced_stats', MetaData(), Column('level_0', BIGINT(), table=<nba_advanced_stats>), Column('Year', BIGINT(), table=<nba_advanced_stats>), Column('Player', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('Pos', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('Age', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('Tm', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('G', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('MP', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('PER', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('TS%', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Column('3PAr', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_advanced_stats>), Col

### Practice select, insert, update statments with SQLAlchemy

Using core, engine construction

Using ORM, session

In [ ]:
stmt = select(Standings).where(Standings.W > 50, Standings.Year > 2020)
with Session(bind=engine) as session:
    for row in session.execute(stmt):
         print(row)

print(stmt)

In [36]:
row = session.scalars(select(Standings)).first()
row.__table__

Table('nba_standings', MetaData(), Column('index', Integer(), table=<nba_standings>, primary_key=True, nullable=False), Column('Year', BIGINT(), table=<nba_standings>), Column('Team', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('W', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('L', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('W/L%', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('GB', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('PS/G', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('PA/G', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('SRS', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('Playoffs', VARCHAR(collation='SQL_Latin1_General_CP1_CI_AS'), table=<nba_standings>), Column('Losing_season', VA

In [53]:
from sqlalchemy import func, cast
stmt = select(
    ("Team Name: " + Standings.Team).label("team_name"), 
    (Standings.W + " Wins").label("wins")).where(Standings.W > 50).order_by(Standings.W.desc())
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.team_name:<40} {row.wins}")
print(stmt)

Team Name: Golden State Warriors         73 Wins
Team Name: San Antonio Spurs             67 Wins
Team Name: Golden State Warriors         67 Wins
Team Name: Golden State Warriors         67 Wins
Team Name: Miami Heat                    66 Wins
Team Name: Houston Rockets               65 Wins
Team Name: Phoenix Suns                  64 Wins
Team Name: San Antonio Spurs             62 Wins
Team Name: San Antonio Spurs             61 Wins
Team Name: Atlanta Hawks                 60 Wins
Team Name: Oklahoma City Thunder         60 Wins
Team Name: Milwaukee Bucks               60 Wins
Team Name: Oklahoma City Thunder         59 Wins
Team Name: Toronto Raptors               59 Wins
Team Name: Golden State Warriors         58 Wins
Team Name: San Antonio Spurs             58 Wins
Team Name: Toronto Raptors               58 Wins
Team Name: Milwaukee Bucks               58 Wins
Team Name: Boston Celtics                57 Wins
Team Name: Denver Nuggets                57 Wins
Team Name: Los Angel

In [63]:
stmt = select(
    ("Team Name: " + Standings.Team).label("team_name"), 
    (Standings.W + " Wins").label("wins")).where(or_(and_(Standings.W < 40, Standings.Playoffs == 'Y'),
                                                     and_(Standings.W > 45, Standings.Playoffs == 'N'))).order_by(Standings.W.desc())
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.team_name:<40} {row.wins}")
print(stmt)

Team Name: Phoenix Suns                  48 Wins
Team Name: Denver Nuggets                46 Wins
Team Name: Brooklyn Nets                 38 Wins
Team Name: Memphis Grizzlies             38 Wins
Team Name: Milwaukee Bucks               38 Wins
Team Name: Atlanta Hawks                 38 Wins
Team Name: New Orleans Pelicans          36 Wins
Team Name: Boston Celtics                36 Wins
Team Name: Portland Trail Blazers        35 Wins
Team Name: Brooklyn Nets                 35 Wins
Team Name: Washington Wizards            34 Wins
Team Name: Orlando Magic                 33 Wins
SELECT :Team_1 || nba_standings."Team" AS team_name, nba_standings."W" || :W_1 AS wins 
FROM nba_standings 
WHERE nba_standings."W" < :W_2 AND nba_standings."Playoffs" = :Playoffs_1 OR nba_standings."W" > :W_3 AND nba_standings."Playoffs" = :Playoffs_2 ORDER BY nba_standings."W" DESC
